In [677]:
import matplotlib
from matplotlib import font_manager

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpaths = ["C:/Users/kwon3/AppData/Local/Microsoft/Windows/Fonts/NanumGothicBold.ttf"]
font_files = font_manager.findSystemFonts(fontpaths=fontpaths)

for ff in font_files:
    font_manager.fontManager.addfont(ff)

matplotlib.rc('font', family="Malgun Gothic")
matplotlib.rcParams['axes.unicode_minus'] = False

'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [635]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [636]:
# DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
# DATA_PATH

In [637]:
DATA_PATH = "data/"
DATA_PATH

'data/'

- 시드값

In [638]:
SEED = 42

- 데이터 불러오기

In [639]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [640]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2310), (12225, 2310))

# 결측치 처리

In [641]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

Series([], dtype: int64)

In [642]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

Series([], dtype: int64)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [643]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2309), (12225, 2309))

- 추가 피처 만들어 보기

In [644]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_지점코드_") ]

train_ft["지점코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["지점코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["지점코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["지점코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["지점코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["지점코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2312), (12225, 2312))

In [645]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_중분류_") ]

train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2315), (12225, 2315))

In [646]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_대분류_") ]

train_ft["대분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["대분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["대분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["대분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["대분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["대분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2318), (12225, 2318))

In [647]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_브랜드코드_") ]

train_ft["브랜드코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["브랜드코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["브랜드코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["브랜드코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["브랜드코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["브랜드코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2321), (12225, 2321))

In [648]:
cols = [ col for col in train_ft.columns if col.endswith("요일_구매비율") ]

train_ft["요일별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["요일별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["요일별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["요일별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["요일별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["요일별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2324), (12225, 2324))

In [649]:
cols = [ col for col in train_ft.columns if col.endswith("월_구매비율") ]

train_ft["월별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["월별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["월별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["월별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["월별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["월별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2327), (12225, 2327))

In [650]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2330), (12225, 2330))

In [651]:
cols = [ col for col in train_ft.columns if col.endswith("분기_구매비율") ]

train_ft["분기별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["분기별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["분기별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["분기별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["분기별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["분기별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2333), (12225, 2333))

In [652]:
cols = [ col for col in train_ft.columns if col.endswith("년_구매비율") ]

train_ft["년도별_구매비율_std"] = train_ft[cols].std(axis=1)
test_ft["년도별_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2334), (12225, 2334))

## Feature Encoding

In [653]:
train_ft["첫구매날짜"] = pd.to_datetime(train_ft["첫구매날짜"])
train_ft["마지막구매날짜"] = pd.to_datetime(train_ft["마지막구매날짜"])
test_ft["첫구매날짜"] = pd.to_datetime(test_ft["첫구매날짜"])
test_ft["마지막구매날짜"] = pd.to_datetime(test_ft["마지막구매날짜"])

train_ft["첫구매년도"] = train_ft["첫구매날짜"].dt.year
train_ft["첫구매월"] = train_ft["첫구매날짜"].dt.month
train_ft["마지막구매년도"] = train_ft["마지막구매날짜"].dt.year
train_ft["마지막구매월"] = train_ft["마지막구매날짜"].dt.month

test_ft["첫구매년도"] = test_ft["첫구매날짜"].dt.year
test_ft["첫구매월"] = test_ft["첫구매날짜"].dt.month
test_ft["마지막구매년도"] = test_ft["마지막구매날짜"].dt.year
test_ft["마지막구매월"] = test_ft["마지막구매날짜"].dt.month

train_ft.shape, test_ft.shape

((14940, 2338), (12225, 2338))

- 범주형 피처
    - 주구매지점, 주구매_중분류, 주구매_대분류, 주구매시간대, 선호브랜드코드, 고객등급

In [654]:
# %pip install category_encoders

In [655]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
oh_enc_cols = ["주구매지점","주구매_대분류", "주구매시간대"]
oh_enc_data = pd.concat([train_ft[oh_enc_cols], test_ft[oh_enc_cols]])
oh_enc.fit(oh_enc_data)

train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[oh_enc_cols]).toarray()

train_ft.shape, test_ft.shape

((14940, 2379), (12225, 2379))

In [656]:
ord_enc = OrdinalEncoder(categories=[['Normal', 'Black', 'Gold', 'Platinum', 'Diamond']])
train_ft["고객등급"] = ord_enc.fit_transform(train_ft[["고객등급"]])
test_ft["고객등급"] = ord_enc.transform(test_ft[["고객등급"]])

train_ft.shape, test_ft.shape

((14940, 2379), (12225, 2379))

In [657]:
train_ft['선호브랜드코드'] = train_ft['선호브랜드코드'].astype('category')
test_ft['선호브랜드코드'] = test_ft['선호브랜드코드'].astype('category')

combined_brands = pd.concat([train_ft['선호브랜드코드'], test_ft['선호브랜드코드']])

enc = ce.CountEncoder()
train_ft["선호브랜드코드_cnt"] = enc.fit_transform(train_ft[['선호브랜드코드']])
test_ft["선호브랜드코드_cnt"] = enc.transform(test_ft[['선호브랜드코드']])

train_ft.shape, test_ft.shape

((14940, 2380), (12225, 2380))

In [658]:
combined_categories = pd.concat([train_ft['주구매_중분류'], test_ft['주구매_중분류']])

enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 2381), (12225, 2381))

- 문자열 피처 삭제

In [659]:
cols = train_ft.select_dtypes(["object", "datetime"]).columns.tolist()
cols

['첫구매날짜', '마지막구매날짜', '주구매지점', '주구매_중분류', '주구매_대분류']

In [660]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 2376), (12225, 2376))

In [661]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [662]:
train_ft.select_dtypes("datetime").columns, test_ft.select_dtypes("datetime").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [663]:
train_ft.isin([np.inf, -np.inf]).any().sum(), test_ft.isin([np.inf, -np.inf]).any().sum()

(0, 0)

In [664]:
train_ft.isna().sum().sum(), test_ft.isna().sum().sum()

(0, 0)

In [665]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, Normalizer
scaler = Normalizer()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,총방문일수,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,평균구매요일,주구매요일,월요일_구매비율,...,주구매시간대_13,주구매시간대_14,주구매시간대_15,주구매시간대_16,주구매시간대_17,주구매시간대_18,주구매시간대_19,주구매시간대_20,선호브랜드코드_cnt,주구매_중분류_cnt
0,0.000002,0.000044,4.882933e-06,3.487809e-07,2.712741e-07,3.966474e-06,1.813661e-05,5.231714e-07,3.487809e-07,1.245646e-08,...,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,1.743905e-07,4.115615e-05
1,0.000004,0.000062,2.808654e-06,1.755409e-07,3.510817e-07,2.060698e-06,5.266226e-07,4.054158e-07,5.266226e-07,8.359088e-09,...,0.0,0.0,0.000000e+00,0.000000e+00,0.0,1.755409e-07,0.0,0.0,1.237739e-03,9.145679e-05
2,0.000002,0.000014,2.406774e-07,4.412419e-07,7.439119e-08,2.013662e-07,1.604516e-07,9.359676e-08,0.000000e+00,1.022486e-08,...,0.0,0.0,4.011290e-08,0.000000e+00,0.0,0.000000e+00,0.0,0.0,3.209032e-07,5.615805e-07
3,0.000002,0.000008,7.147265e-08,4.526601e-07,4.897200e-08,7.474003e-08,2.382422e-07,7.340434e-08,9.529686e-08,1.802914e-09,...,0.0,0.0,2.382422e-08,0.000000e+00,0.0,0.000000e+00,0.0,0.0,5.122206e-06,1.112353e-04
4,0.000003,0.000037,1.527952e-06,7.052085e-07,1.763021e-07,1.406037e-06,5.171529e-06,4.538147e-07,4.701390e-07,9.794563e-09,...,0.0,0.0,0.000000e+00,1.175348e-07,0.0,0.000000e+00,0.0,0.0,8.287375e-04,2.408287e-04


# 정답 데이터

In [666]:
target = train_target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

# 모델 선택

In [669]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
model.fit(train_ft, target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.046933 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 169415
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1689
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# cv 점수 확인해보기

In [559]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [684]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

0.7251038972694498

# 모델 학습

In [561]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.042154 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 169415
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1689
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# 테스트 데이터 예측

In [562]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [563]:
# pred = model.predict_proba(test_ft)[:,1]
# pred

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [564]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [565]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


- 예측 결과를 csv 파일로 저장하여 제출

In [566]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)